<a href="https://colab.research.google.com/github/wittysean/COMPUTER-PROGRAMMING-AND-APPLICATION-113-2/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 依使用者輸入：confirm, buttons, carousel, imgCarousel，分別傳送確認模板、按鈕模板、輪播模板及圖片輪播模板。
- 模板上的按鈕能觸發不同類型的 Action Object。

In [2]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn, URIAction
)

# 讀取 secret
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
LINE_CHANNEL_ACCESS_TOKEN = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
LINE_CHANNEL_SECRET = userdata.get('LINE_CHANNEL_SECRET')

app = Flask(__name__)

configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text

        if action == 'confirm':
            template = ConfirmTemplate(
                text='你喜歡 Charlie Puth 的音樂嗎？',
                actions=[
                    MessageAction(label='喜歡', text='我超喜歡 Charlie Puth'),
                    MessageAction(label='還好', text='其實我也滿喜歡，只是裝作冷靜而已')
                ]
            )
            reply = TemplateMessage(
                alt_text='這是確認視窗',
                template=template
            )

        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/en/0/04/Charlie_Puth_-_Voicenotes.png',
                        title='Voicenotes',
                        text='2018年發行，代表作 Attention 來自此專輯',
                        actions=[
                            URIAction(label='維基百科', uri='https://en.wikipedia.org/wiki/Voicenotes'),
                            MessageAction(label='我喜歡 Voicenotes', text='Voicenotes 是我最愛的專輯')
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/en/e/e0/Charlie_Puth_-_Charlie.png',
                        title='CHARLIE',
                        text='2022年發行，更加展現個人風格',
                        actions=[
                            URIAction(label='維基百科', uri='https://en.wikipedia.org/wiki/Charlie_(Charlie_Puth_album)'),
                            MessageAction(label='我喜歡 CHARLIE', text='CHARLIE 是我最愛的專輯')
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text='這是輪播視窗',
                template=carousel_template
            )

        else:
            reply = TextMessage(text='Please type "confirm" or "carousel"!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

if __name__ == "__main__":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(5000).public_url
    print("👉 請把這個網址填到 LINE Developer Webhook URL：")
    print(f"{public_url}/callback")
    app.run(host="0.0.0.0", port=5000)

👉 請把這個網址填到 LINE Developer Webhook URL：
https://4cde-104-198-26-38.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:25] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:33] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:38] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:42] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:49] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:36:57] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:37:05] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 16:37:08] "POST /callback HTTP/1.1" 200 -
